In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
import pandas as pd

sys.path.append("/Users/jonathan/wine_analysis_hplc_uv/")
sys.path

In [ ]:
from google_sheets_api import post_df_as_sheet_values

path_to_certs = os.path.join(os.getcwd(), "..")
path_to_certs = os.path.join(path_to_certs, "agilette/modules/credientals_tokens")
sheet_id = "15S2wm8t6ol2MRwTzgKTjlTcUgaStNlA22wJmFYhcwAY"
sheet_range = "wine_deg_sequence_table!A1:H500"
print(path_to_certs)

In [ ]:
import datetime

start_time = datetime.datetime.now().replace(
    hour=9, minute=0, second=0, microsecond=0
) + datetime.timedelta(days=1)
end_time = start_time + datetime.timedelta(days=14, hours=8)

time_list = []
while start_time < end_time:
    time_list.append(start_time)
    start_time += datetime.timedelta(hours=1)

In [ ]:
seq_table = pd.DataFrame()

ambient_wines = ["a0101", "a0201", "a0301"]

seq_table["time"] = time_list
seq_table["weekday"] = seq_table["time"].dt.day_name()
seq_table["experiment"] = pd.Series(ambient_wines * 9)
seq_table["rep_number"] = ""
seq_table["runner"] = ""
seq_table.head()

Every Tuesday and Friday the instrument will be reserved for library runs.

In [ ]:
seq_table.loc[
    (seq_table["time"].dt.time >= datetime.time(9, 00, 00))
    & ((seq_table["weekday"] == "Tuesday") | (seq_table["weekday"] == "Friday")),
    "experiment",
] = "library runs"
seq_table.loc[39:50, :]

In [ ]:
import numpy as np


def planning_df_genner():
    planning_df = pd.DataFrame()

    planning_df["weekdays"] = pd.date_range("2023-04-12", "2023-04-27")
    planning_df["day"] = planning_df["weekdays"].dt.day_name()
    planning_df["weekday_num"] = planning_df["weekdays"].dt.weekday

    planning_df.loc[
        (
            (planning_df.weekdays == "2023-04-12")
            | (planning_df.weekdays.dt.day % 2 == 0)
        ),
        "amb_run",
    ] = "y"
    planning_df["amb_run_cumsum"] = planning_df["amb_run"].apply(
        lambda x: 1 if x == "y" else 0
    )
    planning_df["amb_run_cumsum"] = planning_df["amb_run_cumsum"].cumsum()

    planning_df.loc[
        ~(planning_df.weekdays.dt.date == datetime.date(2023, 4, 12))
        & (planning_df.weekdays.dt.day % 2 != 0),
        "n_freezer_run",
    ] = "y"

    planning_df["n_freezer_cumsum"] = planning_df["n_freezer_run"].apply(
        lambda x: 1 if x == "y" else 0
    )
    planning_df["n_freezer_cumsum"] = planning_df["n_freezer_cumsum"].cumsum()

    planning_df.loc[
        ~(planning_df.weekdays.dt.date == datetime.date(2023, 4, 12))
        & (planning_df.weekdays.dt.day % 2 != 0),
        "e_freezer_run",
    ] = "y"

    planning_df["exp_day_num"] = np.arange(1, 17)

    return planning_df


planning_df = planning_df_genner()
planning_df

On a 'experiment' day, we'd have 3 ambient runs, 3 e_freezer runs and 3 n_freezer runs. Now the ambient can be sequenced, but the e_freezer and n_freezer would need to be done manually in order to account for ambient conditions post defrost. thus should do the freezer runs first.



So ambient runs will be done by Monday 2023-04-24, freezer runs done Wednesday 2023-04-26.

Technically I could schedule the ambient runs to go every day at 6am or something..

In [ ]:
post_df_as_sheet_values(seq_table, sheet_id, sheet_range, path_to_certs)

In [ ]:
n_freezer_runs = ["n0101", "n0201", "n0301"]


def sample_code_gen(exp="str", wines=int, rep=int):
    return [
        f"{exp}{str(wine+1).zfill(2)}{str(rep).zfill(2)}" for wine in range(0, wines)
    ]


samples = sample_code_gen("n", 3, 1) + sample_code_gen("e", 3, 1)
samples

wines run from 1 to 3. reps run from 1 - 7 for n and e experiments, 3 for ambient.

In [ ]:
freezer_run_sched = pd.DataFrame()

freezer_run_sched["time"] = pd.date_range(
    start="09:00:00", end="17:00:00", freq="H"
).strftime("%H:%M")
freezer_run_sched.loc[0:5, "samples"] = samples
freezer_run_sched

In [ ]:
planning_df

14 days, 9 hours a day. How many rows?

In [ ]:
14 * 9

In [ ]:
planning_df[planning_df["n_freezer_run"] == "y"]["weekdays"].shape

In [ ]:
126 / 8

In [ ]:
weekdays = pd.concat(
    [planning_df[planning_df["n_freezer_run"] == "y"]["weekdays"]] * 16
).reset_index(drop=True)
weekdays.shape

In [ ]:
seq = freezer_run_sched["time"]
hours = pd.concat([seq] * 14, axis=0).reset_index(drop=True)
hours

In [ ]:
hour_schedule = pd.concat([weekdays, hours], axis=1, ignore_index=True)[:-2]
hour_schedule.columns = ["date", "hour"]
hour_schedule

In [ ]:
planning_df

7 normal and extreme wine repeats, 3 wines each = 21 individual elements.
n0101
n0102
n0201
etc.

In [ ]:
sample_code_gen("n", 3, 7)

Need to generate blocks of nnneee then assign them to days corresponding to n_freezer_run from 9am to 3pm.

In [ ]:
freezer_run_dates = planning_df.loc[planning_df["n_freezer_run"] == "y", "weekdays"]
freezer_run_dates

In [ ]:
def sample_code_gen(exp=[], num_wines=int, num_rep=int):
    exp = pd.DataFrame(exp, columns=["exp"])
    wine = pd.DataFrame(
        [str(wine + 1).zfill(2) for wine in range(0, num_wines)], columns=["wine"]
    )
    reps = pd.DataFrame(
        [str(rep + 1).zfill(2) for rep in range(0, num_rep)], columns=["rep"]
    )

    exp["key"] = 1
    reps["key"] = 1
    wine["key"] = 1

    result = pd.merge(exp, reps, on="key")
    result = pd.merge(result, wine, on="key")[["exp", "rep", "wine"]]

    result["code"] = result["exp"] + result["wine"] + result["rep"]
    return result


codes = sample_code_gen(["a", "n", "e"], 3, 7)
codes.head()

In [ ]:
codes[(codes.exp.isin(["n", "e"])) & (codes["rep"].astype("int") == 1)]

In [ ]:
# hour_schedule.loc[hour_schedule['date'].isin(freezer_run_dates), 'exp'] =

In [ ]:
planning_df

In [ ]:
hour_schedule.sort_values(["date", "hour"])

In [ ]:
hour_schedule["day_num"] = hour_schedule["date"].dt.weekday
hour_schedule["day"] = hour_schedule["date"].dt.day_name()
hour_schedule